In [7]:
import PySimpleGUI as sg
import pandas as pd
import numpy as np
import pickle
from PIL import Image, ImageOps
import webbrowser
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import PySimpleGUI as sg
import tkinter as tk
from matplotlib.backends.backend_tkagg import FigureCanvasAgg
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import xgboost as xgb
from optimized_hyperparameters_hybrid_ML_USED import *  # Ensure this file exists and contains your model's hyperparameters
from scipy.interpolate import make_interp_spline

from PIL import Image, ImageTk

def draw_figure(canvas, figure):
    figure_canvas_agg = FigureCanvasAgg(figure)
    figure_canvas_agg.draw()
    
    buf = figure_canvas_agg.buffer_rgba()
    image = Image.frombuffer('RGBA', (buf.shape[1], buf.shape[0]), buf, 'raw', 'RGBA', 0, 1)
    photoimage = ImageTk.PhotoImage(image=image)
    
    canvas.create_image(0, 0, image=photoimage, anchor=tk.NW)
    return photoimage


# Load the training data
data = pd.read_excel("data.xlsx", sheet_name='data')
data.columns = ['Vf', 'fy', 'fco', 'rohsy']
x_train = pd.read_excel("data.xlsx", sheet_name='x_train')
x_train.columns = ['Vf', 'fy', 'fco', 'rohsy']
y_train = pd.read_excel("data.xlsx", sheet_name='y_train')
resp = pd.read_excel("data.xlsx", sheet_name='resp')

# Define groups of response variables
response_groups = [(0, 5), (6, 6), (7, 16), (17, 17), (18, 18)]

# Normalize function
def normalize(input_df, data_df):
    normalized_data = (input_df - data_df.min()) / (data_df.max() - data_df.min())
    return normalized_data

# Denormalization function
def denormalize(predictions, feature_name, resp_df):
    min_val = resp_df[feature_name].min()
    max_val = resp_df[feature_name].max()
    return predictions * (max_val - min_val) + min_val

# Function to calculate sr values for a given row in all_pred
def calculate_sr(row):
    peak_sr = row['ecc']
    ultimate_sr = row['ecu']
    fractions_before_peak = np.linspace(0.4, 0.9, 6) * peak_sr
    points_before_peak_sr = fractions_before_peak
    fractions_after_peak = np.linspace(1, 10, 10) * (ultimate_sr - peak_sr) / 10 + peak_sr
    points_after_peak_sr = fractions_after_peak
    sr_values = pd.Series(points_before_peak_sr.tolist() + [peak_sr] + points_after_peak_sr.tolist())
    return sr_values
def delete_figure_agg(canvas, figure_agg):
    canvas.delete("all")

# Define the layout for input features
left_col = [
    [sg.Text("Enter Vf:"), sg.InputText(key='-VF-')],
    [sg.Text("Enter fy:"), sg.InputText(key='-FY-')],
    [sg.Text("Enter fco:"), sg.InputText(key='-FCO-')],
    [sg.Text("Enter rohsy:"), sg.InputText(key='-ROHSY-')],
    [sg.Button("Predict and Plot")]
]

# Define the layout for plot
right_col = [
    [sg.Canvas(key='-CANVAS-', size=(450, 340))]  # Adjust size as needed
]


# Define the parameters
parameters = ['Vf', 'fy', 'fco', 'rohsy']

# Initialize the range dictionary
ranges = {}

# Calculate the min and max for each parameter
for param in parameters:
    min_values = [data[param].min()]
    max_values = [data[param].max()]
    
    ranges[param] = [round(max(min_values), 4), round(min(max_values),4)]

# print(ranges)

import PySimpleGUI as sg

sg.theme('DefaultNoMoreNagging')


def check_value(value, range):
    try:
        float_value = float(value)
        if range[0] <= float_value <= range[1]:
            return True
    except ValueError:
        pass
    return False
def OpenLink(url):
    webbrowser.open_new(url)

def create_link_label(text, url):
    root = tk.Tk()
    root.withdraw()
    label = tk.Label(root, text=text, fg="blue", cursor="hand2")
    label.pack()
    label.bind("<Button-1>", lambda event: handle_link_click(url))
    return label
    
layout = [
    [sg.Text('Define the input parameters', text_color='blue', font=(''))],
    [
        sg.Column(layout=[
            [sg.Frame(layout=[
                [sg.Text('Transverse reinforcement ratio, rohsy (%)', size=(38, 1)), sg.Input(key='-ROHSY-', size=(15, 1), enable_events=True)],
                [sg.Text('Yield strength of transverse reinforcement, fy (MPa)', size=(38, 1)), sg.Input(key='-FY-', size=(15, 1), enable_events=True)],
                [sg.Text('Compressive strength of unconfined UHPC, fco (MPa)', size=(38, 1)), sg.Input(key='-FCO-', size=(15, 1), enable_events=True)],
                [sg.Text('Fiber volumetric ratio, Vf (%)', size=(38, 1)), sg.Input(key='-VF-', size=(15, 1), enable_events=True)]],
            title='Input parameters')], 
        ], justification='left'),

        sg.Column(layout=[
            [sg.Frame(layout=[
                [sg.Text(f'{ranges["rohsy"][0]} ≤ rohsy ≤ {ranges["rohsy"][1]}')],
                [sg.Text(f'{ranges["fy"][0]} ≤ fy ≤ {ranges["fy"][1]}')],
                [sg.Text(f'{ranges["fco"][0]} ≤ fco ≤ {ranges["fco"][1]}')],
                [sg.Text(f'{ranges["Vf"][0]} ≤ Vf ≤ {ranges["Vf"][1]}')]],
            title='Range of applications of the model')],             
        ], justification='center')
    ],

    [
        sg.Button('Predict and Plot'), sg.Button('Export to Excel'),sg.Button('Cancel')       
        
        
    ],
    [sg.Text('')],    
    
    [sg.Text('Predicted stress-strain response of confined UHPC', text_color='blue', font=('Helvetica', 12))],
    [

        sg.Column(layout=[
#             [sg.Image(filename=scaled_img_filename, key='-fig1-')],
            [sg.Canvas(key='-CANVAS-', size=(450, 280))],          
            
        ]
                  , justification='center'
                 )
    ],
]

# Open the images
img2 = Image.open('image2.png')
img4 = Image.open('image4.png')

# Get the minimum width and height among the images
widths = [img2.width, img4.width]
heights = [img2.height,  img4.height]
min_width = min(widths)
min_height = min(heights)

# Resize the images to the minimum size
img2 = ImageOps.fit(img2, (min_width, min_height))
img4 = ImageOps.fit(img4, (min_width, min_height))

# Define the scale factor
scale_factor = 0.25

# Resize the images
img2 = img2.resize((int(min_width * scale_factor), int(min_height * scale_factor)))
img4 = img4.resize((int(min_width * scale_factor), int(min_height * scale_factor)))

# Save the resized images
img2.save('image22.png')
img4.save('image44.png')

# To add figures in two columns
fig2 = sg.Image(filename='image22.png', key='-fig2-', size=(min_width * scale_factor, min_height * scale_factor))
fig4 = sg.Image(filename='image44.png', key='-fig4-', size=(min_width * scale_factor, min_height * scale_factor))

# To add description of the image
layout += [
    [sg.Text('')],
    [sg.Column([
    [sg.Text('Authors: Tadesse G. Wakjira and M. Shahria Alam'+ '\n'
             '             The University of British Columbia, Okanagan')],
    [
#         sg.Button('Contact: tgwakjira@gmail.com', key='EMAIL', button_color=('blue', 'gray')),
     sg.Button('www.tadessewakjira.com/Contact', key='WEBSITE', button_color=('white', 'gray')),
     sg.Button('https://alams.ok.ubc.ca', key='WEBSITE', button_color=('white', 'gray')),
    ]
    ],
    element_justification='left'
    ),
        sg.Column(
            [   [fig2,
                fig4,
                ],
            ],
            element_justification='center'
        ),
    ],

    [sg.Text("   If you utilize this software for your work, we kindly request that you cite the corresponding paper as a reference.", 
             size=(90, 1), 
             border_width=1, 
             relief=sg.RELIEF_SUNKEN, 
             background_color='white',
             text_color='black',
             font=('Helvetica', 8, 'bold'))],
]


window = sg.Window('HAI-CONCise-UHPC: Hybrid Artificial Intelligence-based CONstitutive model for Confined UHPC', layout)


# window.close()
predictions_made = False
while True:
    event, values = window.read()

    if event == sg.WIN_CLOSED or event == 'Exit':
        break

    elif event == 'Predict and Plot':
        input_data1 = np.array([
            float(values['-VF-']),
            float(values['-FY-']),
            float(values['-FCO-']),
            float(values['-ROHSY-'])
        ]).reshape(1, -1)

        input_data = pd.DataFrame(input_data1, columns=['Vf', 'fy', 'fco', 'rohsy'])
        input_data = normalize(input_data, data)

        all_predictions = []

        for group_index, (start_col, end_col) in enumerate(response_groups):
            # Extract hyperparameters for each model in this group
            xgb_params = {key: eval(f"xgb_{key}_group{group_index + 1}") for key in ['n_estimators', 'max_depth', 'learning_rate', 'subsample', 'colsample_bytree', 'random_state']}
            gb_params = {key: eval(f"gb_{key}_group{group_index + 1}") for key in ['n_estimators', 'max_depth', 'learning_rate', 'subsample', 'random_state']}
            rf_params = {key: eval(f"rf_{key}_group{group_index + 1}") for key in ['n_estimators', 'max_depth', 'random_state']}
            gb_weight = eval(f"gb_weight_group{group_index + 1}")
            rf_weight = 1.0 - gb_weight

            for col in range(start_col, end_col + 1):
                y_train_col = y_train.iloc[:, col]

                # Train XGBoost
                xgb_model = xgb.XGBRegressor(**xgb_params).fit(x_train, y_train_col)
                xgb_pred = xgb_model.predict(input_data)

                # Calculate Residuals for Gradient Boosting and RandomForest
                residuals_train = y_train_col - xgb_model.predict(x_train)

                # Train Gradient Boosting on Residuals
                gb_model = GradientBoostingRegressor(**gb_params).fit(x_train, residuals_train)
                gb_pred = gb_model.predict(input_data)

                # Train RandomForest on Residuals
                rf_model = RandomForestRegressor(**rf_params).fit(x_train, residuals_train)
                rf_pred = rf_model.predict(input_data)

                # Final Ensemble Prediction
                final_pred = xgb_pred + gb_weight * gb_pred + rf_weight * rf_pred
                all_predictions.append(final_pred[0])

        # Denormalize predictions
        denormalized_predictions = [denormalize(pred, resp.columns[i], resp) for i, pred in enumerate(all_predictions)]
        denormalized_predictions = pd.DataFrame(denormalized_predictions).T
        denormalized_predictions.columns = resp.columns 

        new_sr = denormalized_predictions.apply(calculate_sr, axis=1).values[0]
        new_ss = denormalized_predictions.drop(['ecc', 'ecu'], axis=1).values[0]
        

        # Interpolation for smooth curve
        zero_four_peak_sr = new_sr[0]
        xnew = np.linspace(zero_four_peak_sr, new_sr[-1], 300)
        spl = make_interp_spline(new_sr, new_ss, k=3)
        smooth_ss = spl(xnew)

        fc_value = f"f$_c$ = {input_data1[0][2]} MPa"
        vf_value = f"v$_f$ = {input_data1[0][0]}%"
        roh_t_value = f"ρ$_s$ = {input_data1[0][3]:.3f}%"
        fy_value = f"f$_y$ = {input_data1[0][1]} MPa"

        fig, ax = plt.subplots(figsize=(5, 4))

        ax.plot(xnew, smooth_ss, linestyle='-', label='Predicted', color='red')
        ax.plot([0, zero_four_peak_sr], [0, new_ss[0]], linestyle='-', color='red')
        ax.set_xlim(0, xnew[-1])
        ax.set_ylim(0, np.ceil(max(new_ss) / 50) * 50)
        ax.set_ylabel('Stress (MPa)', fontsize=16)
        ax.set_xlabel('Strain (mm/mm)', fontsize=16)
        ax.text(0.4, 0.6, fc_value, transform=ax.transAxes, ha='left', fontsize=16)
        ax.text(0.4, 0.52, vf_value, transform=ax.transAxes, ha='left', fontsize=16)
        ax.text(0.4, 0.44, roh_t_value, transform=ax.transAxes, ha='left', fontsize=16)
        ax.text(0.4, 0.36, fy_value, transform=ax.transAxes, ha='left', fontsize=16)
        ax.tick_params(axis='both', which='major', labelsize=16)
        ax.legend(loc="lower right", fontsize=16)
        ax.grid(True, linestyle='--')
        plt.tight_layout()
        
        predictions_made = True  # Set this to True after making predictions

        # Replace plt.show() with the integration for PySimpleGUI
        if 'figure_canvas_agg' in globals():
            delete_figure_agg(window['-CANVAS-'].TKCanvas, figure_canvas_agg)

        figure_canvas_agg = draw_figure(window['-CANVAS-'].TKCanvas, fig)


    elif event == 'Export to Excel':
        # Check if predictions have been made
        if predictions_made:
            # Prepend 0 to the arrays
            new_sr = np.insert(new_sr, 0, 0)
            new_ss = np.insert(new_ss, 0, 0)

            # Convert numpy arrays to dataframe
            data_to_save = pd.DataFrame({'Strain': new_sr, 'Stress': new_ss})
                   
            filename = sg.popup_get_file('Save to Excel file', save_as=True, default_extension='.xlsx', file_types=(("Excel Files", "*.xlsx"),))
            
            if filename:
                if not filename.endswith('.xlsx'):
                    filename += '.xlsx'
                data_to_save.to_excel(filename, sheet_name='Predicted Data', index=False, engine='openpyxl')
                sg.popup('Data successfully saved!')
            else:
                sg.popup('No file selected. Data was not saved.')
        else:
            sg.popup('Please predict data first before exporting.')

